In [ ]:
# Imports
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt


# Example parameters for this notebook
data_dir = "../data/random_dyncurves_diffspeeds"

slow_curve_file = f"{data_dir}/slow_curve.parquet"
fast_curve_file = f"{data_dir}/fast_curve.parquet"
damped_curve_file = f"{data_dir}/damped_curve.parquet"

slow_db_file = f"{data_dir}/slow_preprocessed.parquet"
slow_shortpause_db_file = f"{data_dir}/slow_shortpause_preprocessed.parquet"
fast_db_file = f"{data_dir}/fast_preprocessed.parquet"
damped_db_file = f"{data_dir}/damped_preprocessed.parquet"

In [ ]:
def unwrap_curve(curve_df: pl.DataFrame) -> pl.DataFrame:
    # Extract reference curve and the collected one
    return curve_df.explode("curve").with_row_index("time_idx")


def unwrap_csi(df: pl.DataFrame, subcarrier: int) -> pl.DataFrame:
    return (
        df.with_row_index("antenna_capture_num")
        .explode(
            "subcarrier_idxs",
            "csi_abs",
            "csi_phase",
        )
        .filter(pl.col("subcarrier_idxs") == subcarrier)
        .sort("receiver_name")
        .filter((pl.col("receiver_name") != "qca") | (pl.col("csi_abs") < 250))
    )

In [ ]:
example_subc = 3

# -- Read data
slow_curve = pl.read_parquet(slow_curve_file)
fast_curve = pl.read_parquet(fast_curve_file)
damped_curve = pl.read_parquet(damped_curve_file)

slow_df = pl.read_parquet(slow_db_file)
slow_shortpause_df = pl.read_parquet(slow_shortpause_db_file)
fast_df = pl.read_parquet(fast_db_file)
damped_df = pl.read_parquet(damped_db_file)

# -- Unwrap
slow_curve = unwrap_curve(slow_curve)
fast_curve = unwrap_curve(fast_curve)
damped_curve = unwrap_curve(damped_curve)

slow_df = unwrap_csi(slow_df, example_subc)
slow_shortpause_df = unwrap_csi(slow_shortpause_df, example_subc)
fast_df = unwrap_csi(fast_df, example_subc)
damped_df = unwrap_csi(damped_df, example_subc)

In [ ]:
def plot_receiver_timeseries(dataframe, curve, key="csi_abs"):
    fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(16, 12))

    sns.lineplot(
        data=curve.to_pandas(),
        x="time_idx",
        y="curve",
        ax=axs[0, 0],
    ).set(title="Baseline random curve")

    for i, (receiver, group_df) in enumerate(
        dataframe.group_by(["receiver_name"], maintain_order=True)
    ):
        ax = axs.flat[i + 1]

        sns.lineplot(
            x="timestamp",
            y=key,
            hue="subcarrier_idxs",
            data=group_df.to_pandas(),
            ax=ax,
        ).set(title=f"{receiver[0]}")
        ax.legend(loc="lower right")

    plt.tight_layout()
    plt.show()

In [ ]:
plot_receiver_timeseries(slow_df, slow_curve)

In [ ]:
plot_receiver_timeseries(slow_shortpause_df, slow_curve)

In [ ]:
plot_receiver_timeseries(fast_df, fast_curve)

In [ ]:
plot_receiver_timeseries(damped_df, damped_curve)

In [ ]:
asus = (
    damped_df.filter(pl.col("receiver_name").is_in(["asus", "asus2"]))
    .filter(pl.col("csi_abs") > 1550)
    .select("timestamp", "csi_abs", "sequence_number")
)
print(asus)